In [1]:
%reset


In [3]:
import numpy as np
import scipy.io as sio
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.neighbors import KNeighborsClassifier
import cv2
from keras.models import Sequential
from keras.layers.core import Flatten, Dense, Dropout, Activation
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras.layers import Embedding, TimeDistributed, RepeatVector, LSTM
from keras.utils import to_categorical

Using TensorFlow backend.


In [4]:
age1= sio.loadmat('./age.mat')
path1=sio.loadmat('./path.mat')
data_age= age1['data_age']
data_path= path1['data_path']

In [5]:
def vgg_16(weights_path=None):
    model = Sequential()
    model.add(ZeroPadding2D((1,1),input_shape=(224,224,3)))
    model.add(Convolution2D(64, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(64, 3, 3, activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(128, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(128, 3, 3, activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(256, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(256, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(256, 3, 3, activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, 3, 3, activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, 3, 3, activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(Flatten())
    model.add(Dense(4096, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(4096, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(10, activation='softmax'))

    if weights_path:
        model.load_weights(weights_path)

    return model

In [6]:
image_model= vgg_16()

image_model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

/home/carni7vore/anaconda3/envs/Pyhton3_7/lib/python3.6/site-packages/ipykernel_launcher.py:4: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), activation="relu")`
  after removing the cwd from sys.path.
/home/carni7vore/anaconda3/envs/Pyhton3_7/lib/python3.6/site-packages/ipykernel_launcher.py:6: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), activation="relu")`
  
/home/carni7vore/anaconda3/envs/Pyhton3_7/lib/python3.6/site-packages/ipykernel_launcher.py:10: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(128, (3, 3), activation="relu")`
  # Remove the CWD from sys.path while we load stuff.
/home/carni7vore/anaconda3/envs/Pyhton3_7/lib/python3.6/site-packages/ipykernel_launcher.py:12: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(128, (3, 3), activation="relu")`
  if sys.path[0] == '':
/home/carni7vore/anaconda3/envs/Pyhton3_7/lib/python3.6/site-packages/ipykernel_launcher.py:16: Use

/home/carni7vore/anaconda3/envs/Pyhton3_7/lib/python3.6/site-packages/ipykernel_launcher.py:36: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(512, (3, 3), activation="relu")`


In [9]:
train_ptr=0

In [10]:

ori_images=[]
age=[]
path=[]
r=0
# print(data_path[0,0])
[h,w]= data_path.shape
print(w)
list1= range(train_ptr,train_ptr+10000)
for i in list1:
    str_1= ''.join(data_path[0,i])
    str_1='./wiki_crop/'+str_1
    str_1.replace(" \ "," / ")
    
    temp_age= data_age[0,i]
    try:
        data=plt.imread(str_1)   
        
    except:
        print(str_1)
        print(i)
        continue
    img1= cv2.resize(data,(224,224))
    try:
        img= img1.reshape(224*224*3)
    except:
        continue
    ori_images.append(img1)
    age.append(temp_age)
print("finished")
print(len(ori_images))

62328


finished
7952


In [12]:
train_set= ori_images[:6000]
test_set= ori_images[6000:]
train_set=np.asanyarray(train_set)
print(train_set.shape)
print(len(age))
age_train=age[:6000]

len1= train_set.shape[0]
train_label=[]
for i in range(len1):
    temp= age_train[i]
    if temp<0:
        temp=0
    elif temp>99:
        temp=99
    temp= int(temp/10)
    train_label.append(temp)
    
print(train_label[:200])
y_train= to_categorical(train_label,10)

(6000, 224, 224, 3)
7952
[2, 3, 6, 4, 5, 3, 7, 2, 4, 2, 2, 2, 3, 3, 1, 7, 5, 4, 3, 3, 2, 6, 2, 4, 3, 6, 2, 2, 4, 4, 2, 3, 7, 3, 5, 5, 4, 0, 5, 5, 3, 4, 3, 5, 5, 5, 5, 4, 3, 4, 2, 4, 0, 0, 6, 0, 0, 7, 5, 2, 3, 0, 0, 2, 3, 4, 4, 0, 4, 2, 6, 3, 9, 2, 3, 0, 2, 4, 6, 2, 3, 3, 5, 2, 3, 3, 2, 2, 2, 2, 6, 6, 5, 7, 5, 5, 2, 2, 7, 1, 3, 2, 2, 3, 0, 4, 0, 0, 2, 0, 2, 5, 2, 2, 2, 3, 6, 4, 0, 3, 2, 2, 7, 4, 3, 4, 3, 2, 5, 4, 4, 6, 7, 0, 0, 0, 1, 2, 2, 2, 5, 8, 5, 3, 3, 2, 3, 4, 3, 6, 3, 0, 2, 3, 5, 7, 2, 4, 2, 3, 0, 4, 2, 5, 2, 2, 6, 2, 2, 2, 2, 4, 2, 3, 2, 7, 3, 3, 4, 1, 3, 6, 3, 3, 2, 2, 3, 9, 2, 3, 2, 1, 4, 2, 2, 4, 3, 2, 1, 3]


In [13]:
test_set= np.asarray(ori_images[6000:])
len1= test_set.shape[0]
test_label=[]
for i in range(len1):
    temp= age_train[i]
    if temp<0:
        temp=0
    elif temp>99:
        temp=99
    #temp= int(temp/10)
    test_label.append(temp)
    
#y_test= to_categorical(test_label,10)
print(test_set.shape)
#print(test_label[:100])
test_label=np.asanyarray(test_label)
print(test_label.shape)

(1952, 224, 224, 3)
(1952,)


In [14]:
image_model.fit(train_set, y_train,batch_size=4, epochs=4, verbose=2)

Epoch 1/4


 - 335s - loss: 12.5090


Epoch 2/4


 - 328s - loss: 12.5130


Epoch 3/4


In [ ]:
test_out= image_model.predict(test_set,verbose=0)
out= np.argmax(test_out,axis=1)
print(out.shape)
print(out[300:400])

In [ ]:
diff= out*10 - test_label 
plt.hist(train_label,bins=100)
plt.figure()
plt.hist(test_label,bins=100)
plt.figure()
plt.hist(diff,bins=100)
collect_1= [i for i in diff if abs(i)<10]
collect_2= [i for i in diff if abs(i)<15]
collect_3= [i for i in diff if abs(i)<20]
print(len(collect_1)/(len(test_set)))
print(len(collect_2)/len(test_set))
print(len(collect_3)/len(test_set))